In [1]:
import sys 
sys.path.append('..')
from src.blip_lavis import get_image_features, define_model, feature_dim
from src.build_classifier import get_classifier
from src.train_clf import train

/home/omoussa/miniconda3/envs/brn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as FT
import torch

In [3]:
sys.path.append('..')
from cifar.cifarRawCorrupted import get_original_loaders, get_corrupt_loaders

In [4]:
device = 'cuda:0'
model = define_model(device=device)

In [5]:
blip_clf = get_classifier(feature_dim, output_classes=10, n_layers=1).to(device)
train_loader, val_loader, test_loader = get_original_loaders(batch_size=1024, model_name='blip') 
test_corrupt_loader = get_corrupt_loaders(batch_size=1024, model_name='blip')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
c_loader = get_corrupt_loaders(model_name='clip', severity=1)


In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(blip_clf.parameters(), lr=0.001)
n_epochs = 2

In [12]:
losses, accs, val_losses, val_accs = train(model, blip_clf, optim=optim, loss_fn=loss_fn,
                                           train_loader=train_loader, val_loader=val_loader,
                                           feature_fn=get_image_features, epochs=n_epochs, device=device) #TODO resize im in clip transforms

/home/omoussa/miniconda3/envs/brn/lib/python3.9/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


initial loss 2.300713872909546 and initial accuracy 0.11096938699483871
 train loss: 1.6647868871688842, val loss: 1.5183157801628113, Train accuracy 0.8694092035293579, val accuracy 0.9493224024772644 
 train loss: 1.5135070621967315, val loss: 1.5104165911674499, Train accuracy 0.9531494379043579, val accuracy 0.9556999206542969 


In [13]:
torch.save(blip_clf.state_dict(), '../saved_models/blip_clf.pth')


In [14]:
def get_acc(gt, preds = None):
    if preds is not None: 
        return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
        
    
    return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
    

def get_test_acc(emb_model, model, test_loader, feature_fn, device='cuda'):
    eval_acc = []
    eval_losses = []
    for eval_batch in test_loader:
        if len(eval_batch)>2:
            _, ims, labels = eval_batch
        else: 
            ims, labels = eval_batch
        ims, labels = ims.to(device), labels.to(device)
        with torch.no_grad():
            features = feature_fn(emb_model, ims).squeeze()
            preds = model(features)
            val_acc = get_acc(labels.view(-1,), preds)
        
        eval_acc.append(val_acc)
    
    return np.mean(eval_acc)
            # 
test_acc_orig = racc =  get_test_acc(model, blip_clf, test_loader, get_image_features, device=device,)
 
print(test_acc_orig)

0.9548888


In [17]:
corrupts_dict = {}
corrupt_g_acc = []
for cr in ['gaussian_noise', 'speckle_noise', 'impulse_noise', 'shot_noise', ]:
    corrupts_dict[cr] = {}
    for sev in [1, 2, 3, 4, 5]:
        test_loader_corrupt = get_corrupt_loaders(batch_size=1024, corruption_type=cr, severity=sev, model_name='blip')
        acc =  get_test_acc(model, blip_clf, test_loader_corrupt, get_image_features, device=device,)
                                

        corrupts_dict[cr][sev]=acc

In [18]:
corrupts_dict

{'gaussian_noise': {1: 0.79129463,
  2: 0.6651447,
  3: 0.55487686,
  4: 0.511085,
  5: 0.4719368},
 'speckle_noise': {1: 0.86540776,
  2: 0.76840127,
  3: 0.7130361,
  4: 0.6126495,
  5: 0.5202407},
 'impulse_noise': {1: 0.89646643,
  2: 0.8471839,
  3: 0.7957111,
  4: 0.66731507,
  5: 0.55657285},
 'shot_noise': {1: 0.85370696,
  2: 0.7868563,
  3: 0.6453165,
  4: 0.5878428,
  5: 0.4998784}}